In [1]:
import os
import pandas as pd

In [2]:
os.chdir('/nfs/user/MMRec/Amazon_Elec_Dataset')
os.getcwd()

'/nfs/user/MMRec/Amazon_Elec_Dataset'

In [3]:
# load item mapping
i_id_mapping = 'i_id_mapping.csv'
df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (63001, 2)


,asin,itemID
0,0528881469,0
1,0594451647,1
2,0594481813,2
3,0972683275,3


In [4]:
# load item mapping
i_id_mapping = 'i_id_mapping.csv'
df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (63001, 2)


,asin,itemID
0,0528881469,0
1,0594451647,1
2,0594481813,2
3,0972683275,3


In [5]:

import gzip, json
meta_file = 'meta_Electronics.json.gz'

print('0 Extracting U-I interactions.')

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF(meta_file)

print(f'Total records: {meta_df.shape}')
meta_df[:3]

0 Extracting U-I interactions.
Total records: (498196, 9)


,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN


In [6]:
# remapping
map_dict = dict(zip(df['asin'], df['itemID']))

meta_df['itemID'] = meta_df['asin'].map(map_dict)
meta_df.dropna(subset=['itemID'], inplace=True)
meta_df['itemID'] = meta_df['itemID'].astype('int64')
#meta_df['description'] = meta_df['description'].fillna(" ")
meta_df.sort_values(by=['itemID'], inplace=True)

print(f'shape: {meta_df.shape}')
meta_df[:2]

shape: (63001, 10)


,asin,imUrl,description,categories,title,price,salesRank,related,brand,itemID
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN,0
15,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[[Electronics, Computers & Accessories, Touch ...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",NaN,1


In [7]:
ori_cols = meta_df.columns.tolist()

ret_cols = [ori_cols[-1]] + ori_cols[:-1]
print(f'new column names: {ret_cols}')

new column names: ['itemID', 'asin', 'imUrl', 'description', 'categories', 'title', 'price', 'salesRank', 'related', 'brand']


In [8]:
meta_df[:3]

,asin,imUrl,description,categories,title,price,salesRank,related,brand,itemID
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN,0
15,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[[Electronics, Computers & Accessories, Touch ...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",NaN,1
20,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[[Electronics, eBook Readers & Accessories, Po...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,19.65,NaN,"{'also_bought': ['B00AAKLIIS', 'B00A668GUO', '...",Barnes &amp; Noble,2


In [9]:
ret_df = meta_df[ret_cols]
# dump
ret_df.to_csv(os.path.join('./', 'meta-elec.csv'), index=False)
print('done!')

done!


## Reload

In [10]:
indexed_df = pd.read_csv('meta-elec.csv')
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

shape: (63001, 10)


,itemID,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[['Electronics', 'GPS & Navigation', 'Vehicle ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,1,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[['Electronics', 'Computers & Accessories', 'T...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",NaN
2,2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,19.65,NaN,"{'also_bought': ['B00AAKLIIS', 'B00A668GUO', '...",Barnes &amp; Noble
3,3,0972683275,http://ecx.images-amazon.com/images/I/41hYJ9Mw...,The VideoSecu TV mount is a mounting solution ...,"[['Electronics', 'Accessories & Supplies', 'Au...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,{},"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu


In [11]:
## Reload

i_uni = indexed_df['itemID'].unique()

print(f'# of unique items: {len(i_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(i_uni), max(i_uni)))

# of unique items: 63001
min/max of unique learners: 0/63000
